# Genetic Algorihm for UPC&S

## Import packages

In [7]:
from numpy.random import randint
from numpy.random import rand
import pandas as pd
pd.set_option('display.max_row', 500)


## Read Data & Assign the variables

In [6]:
# read data
df_sample = pd.read_csv('sample2.csv')

# assign the variables

cost = df_sample[['item', 'T', 'C']].drop_duplicates().dropna(axis=0)
demand = df_sample[['item', 'T', 'D']].drop_duplicates().dropna(axis=0)
prepaid = df_sample[['item', 'T', 'P']].drop_duplicates().dropna(axis=0)
cycle = df_sample[['item', 'J', 'T', 'M']].drop_duplicates().dropna(axis=0)

cost1 = cost.dropna(axis=0)
demand1 = demand.dropna(axis=0)
prepaid1 = prepaid.dropna(axis=0)
cycle1 = cycle.dropna(axis=0)


ijt_df = pd.DataFrame(df_sample[['item', 'J', 'T']].value_counts()).reset_index()[['item', 'J', 'T']]
ijt_df = ijt_df.sort_values(['item']).reset_index(drop=True)
ijt_set = set()
for index, info in ijt_df.iterrows():
    ijt_set.add((info['item'], info['J'], info['T']))
ijt_set

# variables infomation
T = list(set(df_sample['T']))
I = list(set(df_sample['item']))
J = list(set(df_sample['J']))

# Parameters
cit = {}
pit = {}
dit = {}
mijt = {}

# cit - 단가
for index, info in cost.iterrows():
    cit[info['item'], info['T']] = info['C']
# pit - 선급품/일반품
for index, info in prepaid.iterrows():
    pit[info['item'], info['T']] = info['P']
# dit - 생산목표량
for index, info in demand.iterrows():
    dit[info['item'], info['T']] = info['D']
# mijt - cycle time
for index, info in cycle.iterrows():
    mijt[info['item'], info['J'], info['T']] = info['M']

# adding missing values
for i in I:
    for t in T:
        key = (i, t)
        if key not in cit:
            cit[i, t] = 0
for i in I:
    for t in T:
        key = (i, t)
        if key not in pit:
            pit[i, t] = 0
for i in I:
    for t in T:
        key = (i, t)
        if key not in dit:
            dit[i, t] = 0
for i in I:
    for j in J:
        for t in T:
            key = (i, j, t)
            if key not in mijt:
                mijt[i, j, t] = 0

In [9]:
### 실제 데이터로 바꿔야 합니다
df_raw = pd.read_excel('생산미결리스트 등 샘플데이터.xlsx', sheet_name=None)
machine_bound_info = df_raw['CYCLETIME'][1:].reset_index(drop=True)
item_info = df_raw['생산미결리스트(for cost)'].reset_index(drop=True)
# item-단가 unique 하지 않음 (수량이 다르므로) ==> item 별로 합산을 해서 총 수량을 하기엔... 영업납기일이 다름..! 흠..
# 데이터 자체를 쓰기가.. 흠..demand 부터 뽑는것이 애매함
item_info[['중산도면', '수량']].value_counts()
#item_info[item_info['중산도면']=='057386']

machine_bound_info[['JSDWG', 'MCNO']].value_counts() # 이건 unique함!

machine_bound = {}
for index, info in machine_bound_info.iterrows():
    item = info['JSDWG']
    machine = info['MCNO']
    machine_bound[item, machine] = info['AVG_CT']
item_cost = {}
for index, info in item_info.iterrows():
    item = info['중산도면']
    item_cost[item] = int(info['단가'])
item_demand = {}
for index, info in item_info.iterrows():
    item = info['중산도면']
    item_demand[item] = int(info['수량'])
item_urgent = {}
for index, info in item_info.iterrows():
    item = info['중산도면']
    if info['선급'] != info['선급']:
        item_urgent[item] = 1
    else:
        item_urgent[item] = 0



## Genetic Alogorithm

In [26]:
def dict2bitstring(xijt):
    return list(xijt.values())


def bitstring2dict(bitstring, type='xijt'):
    if type == 'xijt':
        _keys = xijt_keys
    elif type == 'mijt':
        _keys = mijt_keys

    for idx, value in enumerate(bitstring):
        xijt[_keys[idx]] = value
    return xijt


def generation_xijt():
    xijt = {}
    for i in I:
        for j in J:
            for t in T:
                xijt[i, j, t] = randint(0, 300) # 300, 바뀌어야 함
    return xijt


def decode(mijt, xijt):
    for j in J:
        for t in T:
            check = sum(mijt[i, j, t]*xijt[i, j, t] for i in I) <=  600
            if check == False:
                for i in I:
                    if mijt[i, j, t] == 0:
                        xijt[i, j, t] = 0
                    else:
                        xijt[i, j, t] = randint(0, 300)

    for i in I:
        for j in J:
            for t in T:
                if xijt[i, j, t] < 0:
                    xijt[i, j, t] = 0
    return xijt


def constraint_check(xijt):
    #  Constraint 2
    check_list = []
    for j in J:
        for t in T:
            check_value = sum(mijt[i, j, t]*xijt[i, j, t] for i in I) <=  600
            check_list.append(check_value)
    const_2 = all(i==True for i in check_list)
    check = const_2 == True
    penalty = 1 # check_list관련으로 수정되어야 함

    return check, penalty


def objective(xijt):
    uit = {}
    check, penalty = constraint_check(xijt)
    if  check == False:
        return penalty * 100000000
    for i in I:
        for t in T:
            u = dit[i, t] - sum(xijt[i, j, t] for j in J)
            if u >= 0:
                uit[i, t] = u
            else:
                uit[i, t] = 0
    objective = sum(uit[i, t]*cit[i, t]*pit[i, t] for i in I for t in T)
    return objective


# tournament selection
def selection(pop, scores, k=5):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]


def crossover(p1, p2, r_cross):
    p1 = dict2bitstring(p1)
    p2 = dict2bitstring(p2)
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]

    return [c1, c2]


# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]


def genetic_algorithm(objective, bounds, n_iter, n_pop, r_cross, r_mut):
    pop = [generation_xijt() for _ in range(n_pop)]
    best, best_eval = decode(mijt, pop[0]), objective(decode(mijt, pop[0]))
    print(best_eval)

    for gen in range(n_iter):
        decoded = [decode(mijt, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]

        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f'>{gen}, {scores[i]}')

        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]

        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(bitstring2dict(c))
        pop = children
    return [best, best_eval]


In [27]:
xijt = generation_xijt()

# number of generations
n_iter = 150
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(xijt))


xijt_keys = list(xijt.keys())
mijt_keys = list(mijt.keys())

best, score = genetic_algorithm(objective, mijt, n_iter, n_pop, r_cross, r_mut)
print('Done!')


100000000
>1, 0


In [ ]:
best

{('K57791', 416, 1): 173,
 ('K57791', 416, 2): 274,
 ('K57791', 416, 3): 0,
 ('K57791', 416, 4): 0,
 ('K57791', 416, 5): 240,
 ('K57791', 416, 6): 159,
 ('K57791', 422, 1): 0,
 ('K57791', 422, 2): 150,
 ('K57791', 422, 3): 30,
 ('K57791', 422, 4): 167,
 ('K57791', 422, 5): 271,
 ('K57791', 422, 6): 229,
 ('K57791', 424, 1): 20,
 ('K57791', 424, 2): 147,
 ('K57791', 424, 3): 199,
 ('K57791', 424, 4): 178,
 ('K57791', 424, 5): 218,
 ('K57791', 424, 6): 87,
 ('K57791', 425, 1): 147,
 ('K57791', 425, 2): 33,
 ('K57791', 425, 3): 0,
 ('K57791', 425, 4): 55,
 ('K57791', 425, 5): 278,
 ('K57791', 425, 6): 198,
 ('K57791', 426, 1): 0,
 ('K57791', 426, 2): 37,
 ('K57791', 426, 3): 0,
 ('K57791', 426, 4): 91,
 ('K57791', 426, 5): 0,
 ('K57791', 426, 6): 249,
 ('K57791', 401, 1): 196,
 ('K57791', 401, 2): 223,
 ('K57791', 401, 3): 121,
 ('K57791', 401, 4): 19,
 ('K57791', 401, 5): 224,
 ('K57791', 401, 6): 39,
 ('K57791', 434, 1): 271,
 ('K57791', 434, 2): 77,
 ('K57791', 434, 3): 234,
 ('K57791'